In [1]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [2]:
import pandas as pd
import numpy as np
from myUtils import pickle_load, pickle_dump

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# # filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# filename_symbols = path_data_dump + 'my_symbols.csv'  # symbols text file

# _filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_perf_rank_dict =  'perf_rank_dict'  # store symbols from performance rank results
filename_pickled_r_all_ranks =  'r_all_ranks'  # list of top 100 most common symbols from performance rank results
filename_pickled_df_a = 'df_OHLCV_clean'  # df adjusted OHLCV, dropped symbols with no vol and close
filename_pickled_df_c = 'df_close_clean'  # df close, dropped symbols with no vol and close

verbose = False  # True prints more output

#################
# look_back_days = -250 * 60  # subset df iloc days
look_back_days = -250 * 6  # subset df iloc days, 6 years of data
#################

In [3]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [4]:
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
df = df_adjOHLCV[look_back_days::]
df_v = df.xs('Volume', level=1, axis=1)  # select only Volume columns
rows, cols = np.where(df_v == 0)  # row index, column index where trading volumes are zero
idx_no_volume = list(set(cols))
idx_no_volume.sort()
symbols_no_volume = df_v.columns[idx_no_volume]
print(f'symbols with no volume:\n{symbols_no_volume}')

symbols with no volume:
Index(['AMCR', 'AVT', 'BCI', 'BHF', 'BKR', 'CCI', 'CHD', 'CNC', 'CNXC', 'ESAB',
       'FCFS', 'GEN', 'HUBB', 'IBKR', 'JJC', 'PEAK', 'PFG', 'PKI', 'SBNY',
       'SRPT', 'TA', 'UA', 'VC', 'VNT', 'WH', 'XEL'],
      dtype='object')


In [5]:
df_dif = df_v - df_v.shift(periods=1)
rows, cols = np.where(df_dif == 0)
idx_same_volume = list(set(cols))
idx_same_volume.sort()
idx_same_volume
symbols_same_volume = df_v.columns[idx_same_volume]
print(f'symbols with same volume:\n{symbols_same_volume}')

symbols with same volume:
Index(['ACM', 'ACN', 'ADP', 'AGCO', 'AMCR', 'AMED', 'AMG', 'AMGN', 'ANSS',
       'ARW',
       ...
       'VAC', 'VICR', 'VNDA', 'VRSK', 'VSAT', 'WERN', 'WEX', 'WTS', 'WTW',
       'XEL'],
      dtype='object', length=214)


In [6]:
df_c = df.xs('Close', level=1, axis=1)  # select only Close columns
df_c = df_c.fillna(0).copy()  # convert NaNs to zeros
rows, cols = np.where(df_c == 0)  # row index, column index where trading volumes are zero
idx_no_close = list(set(cols))
idx_no_close.sort()
symbols_no_close = df_c.columns[idx_no_close]
print(f'symbols with NaN close:\n{symbols_no_close}')

symbols with NaN close:
Index(['ABB', 'ABNB', 'AIR', 'AIRC', 'ALGM', 'APOG', 'ATCO', 'BHF', 'BJ',
       'BOH', 'BRBR', 'CARR', 'CDAY', 'CEG', 'CHK', 'CHX', 'CNXC', 'CR',
       'CRWD', 'CTVA', 'DDOG', 'DOW', 'DT', 'DTM', 'ENSG', 'ESAB', 'ETH-USD',
       'ETRN', 'FOX', 'FOXA', 'FRC', 'FYBR', 'GDS', 'GEHC', 'GFS', 'GO', 'GXO',
       'HTH', 'JBGS', 'JBL', 'JJC', 'KD', 'LCID', 'MP', 'MRNA', 'MRVL', 'NARI',
       'NTES', 'NVST', 'NVT', 'OGN', 'OTIS', 'PDD', 'PEB', 'PGNY', 'PKI',
       'RIVN', 'SGOV', 'SHC', 'SITM', 'SJR', 'SPR', 'SWAV', 'SWBI', 'TA',
       'TROX', 'TY', 'UAA', 'UNFI', 'VICI', 'VNT', 'VSCO', 'WH', 'WTW', 'YETI',
       'ZM', 'ZS'],
      dtype='object')


In [7]:
symbols_drop = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
print(f'combined symbols with no volume, same volume and no close, inculdes duplicate symbols: {len(symbols_drop)}')
symbols_drop = list(set(symbols_drop))  # drop duplicate symbols
symbols_drop .sort()
df_a = df.drop(symbols_drop, axis=1, level=0)  # drop symbols from OHLCA df
df_c = df_close.iloc[look_back_days::]
df_c = df_c.drop(symbols_drop, axis=1)
print(f'unique symbols dropped from df_a (adjOLHLV) and df_c (Close): {len(symbols_drop)}')

combined symbols with no volume, same volume and no close, inculdes duplicate symbols: 317
unique symbols dropped from df_a (adjOLHLV) and df_c (Close): 294


In [8]:
print(f'symbols with no volume:   {len(symbols_no_volume):>5,}')
print(f'symbols with same volume: {len(symbols_same_volume):>5,}')
print(f'symbols with no close:    {len(symbols_no_close):>5,}\n')
print(f'symbols total before drop:                                        {len(df_close.columns):>5,}')
print(f'unique symbols dropped from df OHLCVA (df_a) and df Close (df_c): {len(symbols_drop):>5,}\n')
print('                                          symbols     rows')
print(f'df adjOHLCV (df_a) after dropped symbols:   {len(df_a.columns)/5:>5,.0f}    {len(df_a):>5,}')
print(f'df Close (df_c) after dropped symbols:      {len(df_c.columns):>5,}    {len(df_c):>5,}')


symbols with no volume:      26
symbols with same volume:   214
symbols with no close:       77

symbols total before drop:                                        1,585
unique symbols dropped from df OHLCVA (df_a) and df Close (df_c):   294

                                          symbols     rows
df adjOHLCV (df_a) after dropped symbols:   1,291    1,500
df Close (df_c) after dropped symbols:      1,291    1,500


In [9]:
pickle_dump(df_a, path_data_dump, filename_pickled_df_a)
print(f'pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_a}')
pickle_dump(df_c, path_data_dump, filename_pickled_df_c)
print(f'pickled df Close after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_c}')

pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCV_clean
pickled df Close after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_close_clean


In [10]:
from myUtils import list_dump

f_symbols_df_close_clean = 'symbols_df_close_clean.csv'  # symbols text file
symbols_df_c = list(df_c)  # column names in df_c
list_dump(symbols_df_c, path_data_dump, f_symbols_df_close_clean)# df_c.columns.to_csv(f_symbols_df_close_clean)

In [11]:
df_a.tail()

A                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
2023-05-18  127.650002  129.330002  126.489998  129.139999  1637000.0   
2023-05-19  129.589996  130.470001  128.619995  128.869995  1480400.0   
2023-05-22  128.660004  129.750000  128.289993  129.089996  1764500.0   
2023-05-23  129.029999  130.539993  127.910004  128.639999  2784400.0   
2023-05-24  115.199997  121.089996  113.280098  120.989998  6832771.0   

                   AA                                              ...  \
                 Open       High        Low      Close     Volume  ...   
Date                                                               ...   
2023-05-18  36.290001  36.930000  35.669998  36.810001  3326300.0  ...   
2023-05-19  37.150002  37.150002  35.930000  36.099998  3544300.0  ...   
2023-05-22  36.180000  36.939999  36.060001  36.400002  3252800.0  ...   
2023-05-23  36.000000  36.169998  35.450001  35.669998  3542500.0  ...   
2023-05-24  35.259998  35.360001  34.205002  34.349998  3396711.0  ...   

                 ZUMZ                                                   ZWS  \
                 Open       High        Low      Close    Volume       Open   
Date                                                                          
2023-05-18  16.780001  17.450001  16.700001  17.389999  265100.0  21.959999   
2023-05-19  17.250000  17.250000  16.020000  16.270000  365900.0  23.100000   
2023-05-22  16.350000  16.410000  16.180000  16.309999  200400.0  22.500000   
2023-05-23  16.200001  16.990000  16.200001  16.760000  245700.0  22.520000   
2023-05-24  17.150000  17.530001  16.840000  17.330000  855753.0  22.280001   

                                                        
                 High        Low      Close     Volume  
Date                                                    
2023-05-18  22.879999  21.930000  22.830000  1622400.0  
2023-05-19  23.150000  22.350000  22.430000  1298800.0  
2023-05-22  22.770000  22.389999  22.709999  1042900.0  
2023-05-23  22.820000  22.270000  22.389999  1405000.0  
2023-05-24  22.295000  21.780001  22.000000  1225615.0  

[5 rows x 6455 columns]

In [12]:
df_c.tail()

,A,AA,AAL,AAP,AAPL,AB,ABBV,ABC,ABM,ABR,...,YY,ZBH,ZBRA,ZD,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2023-05-18,129.139999,36.810001,15.05,121.480003,175.050003,34.889999,143.440002,174.820007,44.250000,12.81,...,28.219999,135.020004,279.329987,64.400002,45.070000,27.240000,29.040001,179.720001,17.389999,22.830000
2023-05-19,128.869995,36.099998,14.82,117.410004,175.160004,34.419998,145.110001,173.389999,43.599998,12.30,...,28.500000,135.000000,280.799988,63.470001,44.520000,26.770000,28.750000,179.550003,16.270000,22.430000
2023-05-22,129.089996,36.400002,14.38,119.440002,174.199997,34.669998,144.899994,172.250000,43.490002,12.51,...,28.219999,135.270004,280.130005,64.629997,44.590000,28.090000,29.040001,179.520004,16.309999,22.709999
2023-05-23,128.639999,35.669998,14.07,116.779999,171.559998,34.139999,142.520004,171.259995,43.599998,12.55,...,28.180000,130.320007,271.220001,63.330002,44.689999,29.389999,28.530001,172.669998,16.760000,22.389999
2023-05-24,120.989998,34.349998,13.80,115.230003,171.839996,33.889999,141.149994,171.059998,42.790001,12.22,...,27.820000,128.619995,261.390015,61.340000,44.450001,28.969999,27.740000,169.139999,17.330000,22.000000
